# Imports

In [ ]:
import os
import glob
import pandas as pd
import numpy as np

import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms
from torch.optim.lr_scheduler import StepLR

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Data

In [ ]:
!wget "https://chmura.put.poznan.pl/s/LxYSsota5PCbXcU/download" -O dataset.zip
!unzip -q dataset.zip

## Preprocessing

In [ ]:
# %%capture
# import os
# import pandas as pd
# import shutil

# source_folder = "./dataset"
# output_folder = "./combined_dataset/images"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# counter = 0

# combined_data = pd.DataFrame(columns=["image", "forward", "left"])

# csv_files = [file for file in os.listdir(source_folder) if file.endswith(".csv")]
# for csv_file in csv_files:
#     csv_file_path = os.path.join(source_folder, csv_file)

#     data = pd.read_csv(csv_file_path, names=["image", "forward", "left"])

#     folder_name = os.path.splitext(csv_file)[0]

#     for _, row in data.iterrows():
#         image_name = f"{int(row['image']):04d}.jpg"
#         old_image_path = os.path.join(source_folder, folder_name, image_name)
#         new_image_name = f"{counter:04d}.jpg"
#         new_image_path = os.path.join(output_folder, new_image_name)

#         shutil.copy2(old_image_path, new_image_path)

#         combined_data = combined_data.append({"image": new_image_name, "forward": row['forward'], "left": row['left']},
#                                              ignore_index=True)
#         counter += 1

# combined_data.to_csv("./combined_dataset/targets.csv", index=False)

In [ ]:
def expandImagePath(img_dir_path, short_name):
    return os.path.join(img_dir_path, str(short_name).zfill(4) + '.jpg')

In [ ]:
shift_sizes = list(range(4))
output_csv_prefix = "./annotations"

output_files = {output_csv_prefix + f"_{shift_size}.csv": shift_size for shift_size in shift_sizes}

dataset_path = "./dataset"

for output_csv_path, shift_size in output_files.items():
    col_names = ['img_name', 'forward', 'left']
    annotations = pd.DataFrame(columns = col_names)

    for csv_path in sorted(glob.glob(os.path.join(dataset_path, '*.csv'))):
        img_dir_path = csv_path.split('.csv')[0]
        df = pd.read_csv(csv_path, names = col_names)
        # Shift data
        df['forward'] = df['forward'].shift(-shift_size)
        df['left'] = df['left'].shift(-shift_size)
        # Remove last rows whithout shifted data
        df = df.drop(df.tail(shift_size).index)
        # Transform image name into full path
        df['img_name'] = df['img_name'].apply(lambda name: expandImagePath(img_dir_path, name))
        # Combine to a single DF
        annotations = pd.concat([annotations, df], axis=0, sort=False)
    # Save to a file
    annotations.to_csv(output_csv_path, index=False)

In [ ]:
output_csv_path = "./annotations_0.csv"

annotations = pd.read_csv(output_csv_path)
annotations.head()

## Loaders

In [ ]:
class DrivingDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        self.annotations = pd.read_csv(csv_path)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        # x
        img_path = self.annotations.iloc[index, 0]
        image = cv2.imread(img_path)
        if self.transform:
            image = self.transform(image)
        # y
        activations =  torch.tensor(self.annotations.iloc[index, 1:], dtype=torch.float32)
        return image, activations

In [ ]:
### AUGMENTATION IS ALSO APPLIED TO VALIDATION, TO AVOID IT, SPLIT DATA BEFORE CREATING DATASET AND CREATE TWO SEPARATE DATASETS

# transform = A.Compose([
#     A.Resize(224, 224),
#     A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.5),
#     A.HueSaturationValue(hue_shift_limit=30, sat_shift_limit=40, val_shift_limit=30, p=0.5),
#     A.GaussNoise(p=0.5),
#     A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.6), p=0.5),
#     A.MotionBlur(p=0.2),
#     A.Normalize(mean=[125.6922, 105.8604, 116.4662], std=[50.3127, 43.3011, 45.1162]),
#     ToTensorV2(),
# ])


transform = torchvision.transforms.ToTensor()


def getDataLoaders(output_csv_path, train_size, BATCH_SIZE):
    dataset = DrivingDataset(output_csv_path, transform) 

    n_examples_train = int(len(dataset) * train_size)
    n_examples_test = len(dataset) - n_examples_train

    train_set, test_set = torch.utils.data.random_split(dataset, [n_examples_train, n_examples_test])

    train_loader = DataLoader(dataset = train_set, batch_size = BATCH_SIZE, shuffle = True)
    test_loader = DataLoader(dataset = test_set, batch_size = BATCH_SIZE, shuffle = False)
    return train_loader, test_loader

### Preview

In [ ]:
output_csv_path = "./annotations_0.csv"

In [ ]:
train_size = 1
BATCH_SIZE = 64

train_loader, test_loader = getDataLoaders(output_csv_path, train_size, BATCH_SIZE)

train_features, train_labels = next(iter(train_loader))

print(f"x batch shape: {train_features.size()}")
print(f"y batch shape: {train_labels.size()}")

img = train_features[0].squeeze().permute(1, 2, 0)
label = train_labels[0]

plt.axis('off')
plt.imshow(img)
plt.show()
print(f"Label: {label}")

# Model

In [ ]:
class DrivingModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc = nn.Linear(8 * 56 * 56, 32)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 8 * 56 * 56)
        x = self.relu3(self.fc(x))
        x = self.tanh(self.fc2(x))
        return x

In [ ]:
class AdaptiveAvgPool2dCustom(nn.Module):
    def __init__(self, output_size):
        super(AdaptiveAvgPool2dCustom, self).__init__()
        self.output_size = np.array(output_size)

    def forward(self, x: torch.Tensor):
        stride_size = np.floor(np.array(x.shape[-2:]) / self.output_size).astype(np.int32)
        kernel_size = np.array(x.shape[-2:]) - (self.output_size - 1) * stride_size
        avg = nn.AvgPool2d(kernel_size=list(kernel_size), stride=list(stride_size))
        x = avg(x)
        return x

In [ ]:
class DeepConvModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(32, 32, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(128, 128, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            AdaptiveAvgPool2dCustom((10, 10))
            # nn.AdaptiveAvgPool2d((10, 10))

        )

        self.fc = nn.Sequential(
            nn.Linear(12800, 32),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, 2),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



model = DeepConvModel()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
class DeepConvModel2(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(3, 8, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(8, 8, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Conv2d(8, 12, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(12, 12, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

        )

        self.fc = nn.Sequential(
            nn.Linear(12*56*56, 16),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(16, 2),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



model = DeepConvModel2()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
class DeepConvModel3(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(

            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(16, 16, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(32, 32, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.AdaptiveAvgPool2d((10, 10))

        )

        self.fc = nn.Sequential(
            nn.Linear(9216, 64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 2),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



model = DeepConvModel3()
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
# models = {"DrivingModel": DrivingModel()}

# for modelName, model in models.items():
#     n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print(f"{modelName} Number of trainable parameters: {n_trainable_params}")

## Training

In [ ]:
def train_step(model, inputs, targets, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    return loss.item()

def val_step(model, inputs, targets, criterion):
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        loss = criterion(outputs, targets)
    return loss.item()


# class Checkpoint:
#     def __init__(self, checkpoint_path = "./model.pt"):
#         self.checkpoint_path = checkpoint_path
#         self.min_monitor_loss = np.inf

#     def checkpoint(self, monitor_loss):
#         if monitor_loss < self.min_monitor_loss:
#             self.min_monitor_loss = monitor_loss
#             torch.save(model.state_dict(), self.checkpoint_path)
#             print(f"\tSaved checkpoint at {self.checkpoint_path}")


# class EarlyStopper:
#     def __init__(self, patience=1, min_delta=0):
#         self.patience = patience
#         self.min_delta = min_delta
#         self.counter = 0
#         self.min_monitor_loss = np.inf

#     def stop(self, monitor_loss):
#         if monitor_loss < self.min_monitor_loss:
#             self.min_monitor_loss = monitor_loss
#             self.counter = 0
#         elif monitor_loss > (self.min_monitor_loss + self.min_delta):
#             self.counter += 1
#             if self.counter >= self.patience:
#                 return True
#         return False


def train_loop(model, checkpoint_path, N_EPOCHS, train_loader, val_loader):
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", DEVICE)
    device = torch.device(DEVICE)
    # model.to(device)

    criterion_mse = nn.MSELoss().to(device)
    criterion_mae = nn.L1Loss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.1, verbose=True)

    # checkpoint = Checkpoint(checkpoint_path)
    # early = EarlyStopper(patience=3)

    train_losses = []
    val_losses = []
    val_losses_mae = []
    n_batches_train = len(train_loader)
    n_batches_val = len(val_loader)

    for epoch in range(N_EPOCHS):
        total_epoch_loss_train = 0.0
        total_epoch_loss_val = 0.0
        total_epoch_loss_val_mae = 0.0
        # Training
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            total_epoch_loss_train += train_step(model, inputs, labels, optimizer, criterion_mse)

        if epoch in (20, 30):
            scheduler.step()

        # Validation
        for i, data in enumerate(val_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            total_epoch_loss_val += val_step(model, inputs, labels, criterion_mse)
            total_epoch_loss_val_mae += val_step(model, inputs, labels, criterion_mae)
        # Storing losses
        avg_epoch_loss_train = total_epoch_loss_train / n_batches_train
        train_losses.append(avg_epoch_loss_train)
        avg_epoch_loss_val = total_epoch_loss_val / n_batches_val
        val_losses.append(avg_epoch_loss_train)
        avg_epoch_loss_val_mae = total_epoch_loss_val_mae / n_batches_val
        val_losses_mae.append(avg_epoch_loss_val_mae)
        print(f"Epoch [{epoch + 1}/{N_EPOCHS}], Loss: {avg_epoch_loss_train:.4f}, Validation loss: {avg_epoch_loss_val:.4f}, Validation loss MAE: {avg_epoch_loss_val_mae:.4f}")
        # Checkpoint
        # checkpoint.checkpoint(avg_epoch_loss_val)
        # Early stopping
        # if early.stop(avg_epoch_loss_val):
        #     print("\tEarly stopping")
        #     break

In [ ]:
train_loader, test_loader = getDataLoaders(output_csv_path, 1, BATCH_SIZE)
N_EPOCHS = 40

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(DEVICE)

model = DeepConvModel2().to(device)

train_loop(model, None, N_EPOCHS, train_loader, test_loader)

In [ ]:
model.eval()
train_features, train_labels = next(iter(test_loader))
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    img = train_features[i].to(device).unsqueeze(0)
    with torch.no_grad():
        y_pred = model(img).cpu().numpy()[0]

    image = img.squeeze().permute(1, 2, 0).cpu().numpy()
    y_true = train_labels[i].numpy()

    y = np.array([y_true, y_pred])

    figure.add_subplot(rows, cols, i)
    plt.title(str(y), fontsize = 10)
    plt.axis("off")
    plt.imshow(image)

In [ ]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(model,
                    dummy_input,
                    './model2.onnx',
                    export_params=True,
                    opset_version=11,
                    do_constant_folding=True,
                    input_names = ['input'],
                    output_names = ['output'])

In [ ]:
# N_EPOCHS = 30

# output_files = {output_csv_prefix + f"_{shift_size}.csv": shift_size for shift_size in shift_sizes}
# models = {output_csv_path: [f"DrivingModel_shift_{shift_size}", DrivingModel()]
#           for output_csv_path, shift_size in output_files.items()}

# for output_csv_path, modelWithName in models.items():
#     modelName, model = modelWithName
#     checkpoint_path = f"./{modelName}.pt"

#     train_loader, test_loader = getDataLoaders(output_csv_path, train_size, BATCH_SIZE)

#     print(modelName)
#     train_loop(model, checkpoint_path, N_EPOCHS, train_loader, test_loader)

## Evaluation

In [ ]:
# def test_loop(model, checkpoint_path, test_loader):
#     DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     print("Using device:", DEVICE)
#     device = torch.device(DEVICE)

#     criterion = nn.MSELoss().to(device)
#     criterion_mae = nn.L1Loss().to(device)

#     n_batches = len(test_loader)

#     # model.load_state_dict(torch.load(checkpoint_path))
#     model.to(device)
#     model.eval()

#     total_loss = 0.0
#     total_loss_mae = 0.0
#     for i, data in enumerate(test_loader):
#         inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         total_loss += val_step(model, inputs, labels, criterion)
#         total_loss_mae += val_step(model, inputs, labels, criterion_mae)
#     avg_loss = total_loss / n_batches
#     avg_loss_mae = total_loss_mae / n_batches
#     print(f"\tValidation loss: {avg_loss:.4f}, Validation loss MAE: {avg_loss_mae:.4f}")

In [ ]:
# output_files = {output_csv_prefix + f"_{shift_size}.csv": shift_size for shift_size in shift_sizes}
# models = {output_csv_path: [f"DrivingModel_shift_{shift_size}", DrivingModel()]
#           for output_csv_path, shift_size in output_files.items()}

# for output_csv_path, modelWithName in models.items():
#     modelName, model = modelWithName
#     checkpoint_path = f"./{modelName}.pt"

#     train_loader, test_loader = getDataLoaders(output_csv_path, train_size, BATCH_SIZE)

#     print(modelName)
#     test_loop(model, checkpoint_path, test_loader)

In [ ]:
# model.eval()
# train_features, train_labels = next(iter(test_loader))
# figure = plt.figure(figsize=(8, 8))
# cols, rows = 3, 3
# for i in range(1, cols * rows + 1):
#     img = train_features[i].to(device).unsqueeze(0)
#     # print(img.shape)
#     with torch.no_grad():
#         y_pred = model(img).cpu().numpy()[0]

#     image = img.squeeze().permute(1, 2, 0).cpu().numpy()
#     y_true = train_labels[i].numpy()

#     y = np.array([y_true, y_pred])

#     figure.add_subplot(rows, cols, i)
#     plt.title(str(y), fontsize = 10)
#     plt.axis("off")
#     plt.imshow(image)

In [ ]:
# dummy_input = torch.randn(1, 3, 224, 224).to(device)
# torch.onnx.export(model,
#                     dummy_input,
#                     './model.onnx',
#                     export_params=True,
#                     opset_version=11,
#                     do_constant_folding=True,
#                     input_names = ['input'],
#                     output_names = ['output'])

In [ ]:
# output_files = {output_csv_prefix + f"_{shift_size}.csv": shift_size for shift_size in shift_sizes}
# models = {output_csv_path: [f"DrivingModel_shift_{shift_size}", DrivingModel()]
#           for output_csv_path, shift_size in output_files.items()}

# train_size = 0.9
# BATCH_SIZE = 32

# output_csv_path = "./annotations_0.csv"
# _, test_loader = getDataLoaders(output_csv_path, train_size, BATCH_SIZE)
# train_features, train_labels = next(iter(test_loader))

# for _, modelWithName in models.items():
#     modelName, model = modelWithName
#     checkpoint_path = f"./{modelName}.pt"
#     model.load_state_dict(torch.load(checkpoint_path))
#     model.eval()

#     figure = plt.figure(figsize=(8, 8))
#     cols, rows = 3, 3
#     for i in range(1, cols * rows + 1):
#         img = train_features[i]
#         with torch.no_grad():
#             y_pred = model(img).numpy()[0]

#         image = img.squeeze().permute(1, 2, 0).numpy()
#         y_true = train_labels[i].numpy()

#         y = np.array([y_true, y_pred])

#         figure.add_subplot(rows, cols, i)
#         plt.title(str(y), fontsize = 10)
#         plt.axis("off")
#         plt.imshow(image)
#     figure.suptitle(modelName)
#     plt.show()

## Saving

In [ ]:
# !pip3 install onnx

# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# output_files = {output_csv_prefix + f"_{shift_size}.csv": shift_size for shift_size in shift_sizes}
# models = {output_csv_path: [f"DrivingModel_shift_{shift_size}", DrivingModel()]
#           for output_csv_path, shift_size in output_files.items()}

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print("Using device:", DEVICE)
# device = torch.device(DEVICE)
# dummy_input = torch.randn(1, 3, 224, 224).to(device)

# for output_csv_path, modelWithName in models.items():
#     modelName, model = modelWithName
#     checkpoint_path = f"./{modelName}.pt"

#     model.load_state_dict(torch.load(checkpoint_path))
#     model.to(device)

#     path = f"gdrive/MyDrive/{modelName}.onnx"
#     torch.onnx.export(model,
#                       dummy_input,
#                       path,
#                       export_params=True,
#                       opset_version=11,
#                       do_constant_folding=True,
#                       input_names = ['input'],
#                       output_names = ['output'])